TIF images → `Extract-Channels` → `Visualize-Inputs` → `Predict-Segments` → `Visualize-Segmentation` → predicted segmentations

# Setup

In [1]:
import os
import timeit

In [2]:
# Parameters (overridden by notebook parameters)
input_channels_path = './saves/input_channels.npz'
model_path = os.path.expanduser('~') + '/.keras/models/MultiplexSegmentation'
output_path = './saves'

### Load Mesmer model & create app

In [3]:
from imaging_helpers import make_app

app = make_app(model_path=model_path)

2023-07-10 15:25:31.047780: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loaded model in 25.463194751995616 s


### Read input files

In [4]:
import numpy as np

with np.load(input_channels_path) as loader:
    # An array of shape [num_inputs, height, width] containing intensity of nuclear channel
    inputs_nuclear = loader['nuclear']

    # An array of shape [num_inputs, height, width] containing intensity of membrane channel
    inputs_membrane = loader['membrane']

# An array of shape [num_inputs, height, width, 2] containing nuclear & membrane channels
input_channels = np.stack((inputs_nuclear, inputs_membrane), axis=-1)

# Generate predictions

This step is expensive!

In [5]:
t = timeit.default_timer()

import logging
logging.root.setLevel(logging.DEBUG)

import tracemalloc
tracemalloc.start()
segmentation_predictions = app.predict(input_channels, image_mpp=0.5)
snapshot = tracemalloc.take_snapshot()
tracemalloc.stop()

print('Prediction finished in %s s' % (timeit.default_timer() - t))

INFO:root:Converting image dtype to float
DEBUG:Mesmer:Pre-processed data with mesmer_preprocess in 53.38774848601315 s
DEBUG:Mesmer:Model inference finished in 927.9249126589857 s
DEBUG:Mesmer:Post-processing results with mesmer_postprocess and kwargs: {'whole_cell_kwargs': {'maxima_threshold': 0.075, 'maxima_smooth': 0, 'interior_threshold': 0.2, 'interior_smooth': 2, 'small_objects_threshold': 15, 'fill_holes_threshold': 15, 'radius': 2}, 'nuclear_kwargs': {'maxima_threshold': 0.1, 'maxima_smooth': 0, 'interior_threshold': 0.2, 'interior_smooth': 2, 'small_objects_threshold': 15, 'fill_holes_threshold': 15, 'radius': 2}, 'compartment': 'whole-cell'}
/Users/davidhaley/dev/deepcell-imaging/notebooks/venv/lib/python3.10/site-packages/deepcell_toolbox/deep_watershed.py:108: UserWarning: h_maxima peak finding algorithm was selected, but the provided image is larger than 5k x 5k pixels.This will lead to slow prediction performance.
  warnings.warn('h_maxima peak finding algorithm was sele

Prediction finished in 1326.915612539975 s


# Save predictions

In [6]:
import linecache
def display_top(snapshot, key_type='lineno', limit=3):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
        tracemalloc.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        # replace "/path/to/module/file.py" with "module/file.py"
        filename = os.sep.join(frame.filename.split(os.sep)[-2:])
        print("#%s: %s:%s: %.1f KiB"
              % (index, filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))


display_top(snapshot, limit=10)

Top 10 lines
#1: <__array_function__ internals>:180: 543818.9 KiB
#2: internal/python_message.py:1501: 1160.9 KiB
    self._parent_message_weakref = weakref.proxy(parent_message)
#3: internal/decoder.py:553: 1017.3 KiB
    value = str(byte_str, 'utf-8')
#4: framework/ops.py:500: 971.3 KiB
    self._consumers = []
#5: internal/python_message.py:1465: 762.4 KiB
    other_field = self._oneofs.setdefault(field.containing_oneof, field)
#6: framework/ops.py:2110: 512.5 KiB
    self._graph = g
#7: python3.10/abc.py:123: 470.4 KiB
    return _abc_subclasscheck(cls, subclass)
#8: internal/python_message.py:1338: 412.0 KiB
    self._fields[field] = value
#9: internal/containers.py:441: 363.6 KiB
    new_element = self._message_descriptor._concrete_class()
#10: python3.10/linecache.py:137: 334.7 KiB
    lines = fp.readlines()
1772 other: 6704.6 KiB
Total allocated size: 556528.5 KiB


In [7]:
import numpy as np
np.savez_compressed(output_path + '/segmentation_predictions.npz', predictions=segmentation_predictions)